In [1]:
from collections import OrderedDict
from collections import Counter
import numpy as np
import pandas
import heapq

In [3]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('train.txt', 'r') as f:
    for line in f.xreadlines():
        visits, purchases = line.strip().split(';')
        
        visits = visits.split(',')
        purchases = purchases.split(',')
        for goods in visits:
            visit_popularity[goods] += 1
        if purchases != '':
            for goods in purchases:
                purchase_popularity[goods] += 1
            
        # TODO
        # update visit_popupurlarity
        # update purchase_popularity
print(visit_popularity)

Counter({'73': 677, '158': 641, '204': 396, '262': 387, '162': 318, '7': 312, '137': 306, '1185': 284, '6': 283, '170': 280, '800': 253, '5202': 227, '8': 225, '609': 213, '3149': 213, '3324': 204, '1346': 199, '751': 197, '1184': 186, '1283': 186, '1844': 186, '1933': 186, '325': 184, '4604': 177, '551': 177, '42149': 174, '1334': 174, '259': 173, '1852': 172, '70238': 170, '258': 167, '72': 167, '758': 166, '1342': 165, '85': 165, '343': 164, '2290': 163, '5501': 160, '1323': 159, '3718': 158, '301': 156, '115': 154, '3697': 153, '2922': 153, '879': 153, '6974': 151, '260': 150, '1214': 149, '3286': 149, '2084': 149, '28': 148, '255': 147, '363': 147, '302': 144, '71': 141, '791': 140, '1349': 140, '1595': 140, '875': 139, '1934': 139, '884': 138, '1814': 138, '2845': 136, '553': 135, '759': 133, '5894': 133, '3882': 131, '1651': 131, '2397': 129, '11027': 129, '1949': 129, '114': 129, '72586': 127, '552': 125, '469': 124, '2287': 122, '1843': 122, '1011': 122, '966': 122, '316': 121

In [4]:
def recommend_by_purchase(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: purchase_popularity.get(x, 0))

def recommend_by_visit(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), key=lambda x: visit_popularity.get(x, 0))

In [14]:
def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = purchases.split(',')
                rec = recommend(visits, max_count)
                
                sessions_count += 1
                avg_recall[len(rec) - 1] += float(len(set(purchases) & set(rec))) / len(purchases)
                avg_precision[len(rec) - 1] += float(len(set(purchases) & set(rec))) / len(rec)
    print(avg_recall)     
    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
    }).set_index('k')


In [16]:

def calculate_metrics(recommend, max_count, sessions_file):
    with open(sessions_file, 'r') as f:
        avg_recall = np.zeros(max_count)
        avg_precision = np.zeros(max_count)
        sessions_count = 0
        for line in f.xreadlines():
            visits, purchases = line.strip().split(';')
            if purchases != '':
                visits = visits.split(',')
                purchases = set(purchases.split(','))
                rec = recommend(visits, max_count)
                rec_hits = np.array(map(lambda x: x in purchases, rec))
                hits = np.zeros(max_count)
                hits[:len(rec_hits)] = rec_hits
                sessions_count += 1
                avg_recall +=  np.cumsum(hits) / len(purchases)
                avg_precision += np.cumsum(hits) / (np.arange(max_count) + 1)
    print(avg_recall)
    return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
    }).set_index('k')

In [17]:
calculate_metrics(recommend_by_purchase, 5, 'train.txt')


[ 2483.92576868  3026.83927184  3218.09920357  3300.27268842  3342.11674714]


,avg_precision@k,avg_recall@k
k,,
1,0.80,0.69
2,0.53,0.84
3,0.39,0.89
4,0.31,0.91
5,0.25,0.93


In [7]:
calculate_metrics(recommend_by_purchase, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.10,0.18
3,0.05,0.11
4,0.03,0.08
5,0.08,0.26


In [8]:
calculate_metrics(recommend_by_visit, 5, 'train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.09,0.17
3,0.05,0.11
4,0.03,0.09
5,0.08,0.26


In [9]:
calculate_metrics(recommend_by_visit, 5, 'test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.20,0.19
2,0.10,0.18
3,0.05,0.11
4,0.03,0.08
5,0.08,0.24


In [27]:
test_arr = np.zeros(5)
test_arr = test_arr / 2
print(test_arr)

a = [1,2,3,3]
b = [3,4,5,1]
c = set(a) & set(b)
print(c)

[ 0.  0.  0.  0.  0.]
set([1, 3])
